# Page 1-6

In [22]:
import time
import random
import re
import datetime
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

# ========== 1. User-Agent 列表（仅桌面端 Chrome） ==========
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.106 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_6_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.106 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.106 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6261.111 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_3_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6261.129 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.185 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.160 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_3_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.160 Safari/537.36",
    "Mozilla/5.0 (X11; Fedora; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6099.234 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.6045.199 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_6_8) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.6045.199 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.5993.88 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.5938.149 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_7_10) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.5938.149 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.5845.141 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.5845.110 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.5790.170 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.133 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.90 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_0_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.106 Safari/537.36"
]
random_user_agent = random.choice(USER_AGENTS)

# ========== 2. 初始化 WebDriver ==========
service = Service("/Users/zhuxiaowen/Desktop/chromedriver-mac-arm64/chromedriver")
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument(f"user-agent={random_user_agent}")
options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(service=service, options=options)
print(f"✅ 启动 WebDriver, 使用UA: {random_user_agent}")

# ========== 3. 访问页面 ==========
base_url = "https://www.target.com/c/facial-cleansers-skin-care-beauty/-/N-5xtzi?moveTo=product-list-grid"
driver.get(base_url)
time.sleep(random.uniform(5, 10))

# ========== 4. 修改点①：等待商品卡片加载完成 ==========
def wait_until_products_loaded(min_count=25, timeout=15):
    try:
        WebDriverWait(driver, timeout).until(
            lambda d: len(d.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")) >= min_count
        )
    except TimeoutException:
        print(f"⚠️ 超时未加载出 {min_count} 个商品，当前页可能不完整")

# ========== 5. 修改点②：强化滚动逻辑，直到商品数量不再增加 ==========
def scroll_and_wait():
    last_height = driver.execute_script("return document.body.scrollHeight")
    prev_count = 0

    for i in range(30):
        print(f"🔄 滚动页面: 第 {i+1} 次")
        driver.execute_script("window.scrollBy(0, 1200);")
        time.sleep(random.uniform(2, 4))

        products = driver.find_elements(By.XPATH, "//a[@data-test='product-title']")
        if len(products) > prev_count:
            prev_count = len(products)
        else:
            print("✅ 页面懒加载完成")
            break

        try:
            if products:
                ActionChains(driver).move_to_element(products[-1]).perform()
        except:
            pass

    time.sleep(3)

# ...前面的 import 和设置不变...

# ========== 🔧 修改点①：记录每一页的首个商品链接 ==========
visited_first_links = set()

product_list = []

def scrape_products():
    global product_list, visited_first_links
    scroll_and_wait()
    wait_until_products_loaded()

    products = driver.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")
    print(f"🔍 当前页面商品数: {len(products)}")

    if not products:
        print("⚠️ 本页无商品，跳过")
        return

    # 🔧 修改点②：判断是否回到了已访问页面（用首个商品链接判重）
    try:
        first_link = products[0].find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
    except:
        first_link = None

    if first_link in visited_first_links:
        print("🔁 检测到页面内容重复，疑似回到了第一页，停止抓取")
        raise StopIteration
    visited_first_links.add(first_link)

    for product in products:
        try:
            title_element = product.find_element(By.XPATH, ".//a[@data-test='product-title']//div")
            name = title_element.text.strip()
        except NoSuchElementException:
            continue

        try:
            brand = product.find_element(By.XPATH, ".//a[@data-test='@web/ProductCard/ProductCardBrandAndRibbonMessage/brand']").text.strip()
        except:
            brand = "N/A"

        try:
            price = product.find_element(By.XPATH, ".//span[@data-test='current-price']").text.strip()
        except:
            price = "N/A"

        try:
            unit_price = product.find_element(By.XPATH, ".//span[@data-test='unit-price']").text.strip()
        except:
            unit_price = "N/A"

        try:
            rating = product.find_element(By.XPATH, ".//span[@data-test='ratings']").text.strip()
        except:
            rating = "N/A"

        try:
            reviews = product.find_element(By.XPATH, ".//span[@data-test='rating-count']").text.strip()
        except:
            reviews = "0 reviews"

        try:
            product.find_element(By.XPATH, ".//div[@data-test='comparison-price']")
            is_on_sale = 1
        except:
            is_on_sale = 0

        try:
            if " - " in price:
                is_free_shipping = "N/A"
            else:
                price_value = float(price.replace("$", ""))
                is_free_shipping = 1 if price_value >= 35 else 0
        except:
            is_free_shipping = "N/A"

        try:
            shipping_info = product.find_element(By.XPATH, ".//div[@data-test='LPFulfillmentSectionShippingFA_standardShippingMessage']").text
            if "get it tomorrow" in shipping_info:
                is_3day_delivery = 1
            else:
                match = re.search(r"arrives\s+\w+,\s+([A-Za-z]+ \d+)", shipping_info)
                if match:
                    delivery_str = match.group(1)
                    delivery_date = datetime.datetime.strptime(f"{delivery_str} {datetime.datetime.now().year}", "%b %d %Y").date()
                    days_diff = (delivery_date - datetime.datetime.now().date()).days
                    is_3day_delivery = 1 if 0 <= days_diff <= 3 else 0
                else:
                    is_3day_delivery = "N/A"
        except:
            is_3day_delivery = "N/A"

        try:
            link = product.find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
        except:
            link = "N/A"

        product_list.append({
            "product_name": name,
            "brand": brand,
            "current_price": price,
            "unit_price": unit_price,
            "star_rating": rating,
            "review_counts": reviews,
            "is_on_sale": is_on_sale,
            "is_free_shipping": is_free_shipping,
            "is_3day_delivery": is_3day_delivery,
            "product_category": "Cleansers",
            "product_link": link
        })

# 翻页逻辑（原样）
def go_to_next_page():
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@data-test='next']"))
        )
        driver.execute_script("arguments[0].click();", next_button)
        print("✅ 点击下一页")
        time.sleep(random.uniform(8, 12))
        return True
    except:
        print("❌ 没有找到下一页按钮，结束爬取")
        return False

# ========== 主循环（加异常捕捉 StopIteration） ==========
max_pages = 6
try:
    for page_num in range(max_pages):
        print(f"\n📄 正在爬取第 {page_num + 1} 页...")
        scrape_products()
        if not go_to_next_page():
            break
except StopIteration:
    print("⛔ 检测到重复页，主动中止爬虫")

driver.quit()
print("✅ WebDriver 关闭")


print(f"📦 共抓取商品（去重后）：{len(product_list)} 条")


✅ 启动 WebDriver, 使用UA: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.5938.149 Safari/537.36

📄 正在爬取第 1 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页

📄 正在爬取第 2 页...
🔄 滚动页面: 第 1 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页

📄 正在爬取第 3 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页

📄 正在爬取第 4 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页

📄 正在爬取第 5 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页

📄 正在爬取第 6 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页
✅ WebDriver 关闭
📦 共抓取商品（去重后）：158 条


In [25]:
df1 = pd.DataFrame(product_list)
df1

product_name           brand  \
0    La Roche Posay Toleriane Purifying Facial Clea...  La Roche Posay   
1    Geologie Clear System Acne Control Face Wash -...        Geologie   
2    CeraVe Foaming Face Wash, Facial Cleanser for ...          CeraVe   
3    La Roche Posay Toleriane Purifying Facial Clea...  La Roche Posay   
4    CeraVe Face Wash, Hydrating Facial Cleanser fo...          CeraVe   
..                                                 ...             ...   
153  NIVEA Men Maximum Hydration Deep Cleaning Face...           NIVEA   
154  Clinique All About Clean Liquid Facial Soap - ...        Clinique   
155  Pixi Skintreats Glow Tonic Glycolic Acid Exfol...            Pixi   
156  Neutrogena Makeup Remover Melting Balm with Vi...      Neutrogena   
157  TULA SKINCARE So Polished Exfoliating Sugar Sc...   TULA SKINCARE   

       current_price           unit_price  \
0     $9.99 - $20.49                  N/A   
1             $18.29  ($4.51/fluid ounce)   
2     $6.49 - $19.69                  N/A   
3     $9.99 - $20.49                  N/A   
4     $6.49 - $19.69                  N/A   
..               ...                  ...   
153            $8.59        ($1.95/ounce)   
154           $28.00        ($4.18/ounce)   
155  $15.00 - $35.49                  N/A   
156           $14.69        ($7.35/ounce)   
157           $34.00       ($11.72/ounce)   

                               star_rating  review_counts  is_on_sale  \
0     4.7 out of 5 stars with 6455 ratings   6455 reviews           0   
1      4.8 out of 5 stars with 240 ratings    240 reviews           0   
2    4.7 out of 5 stars with 13334 ratings  13334 reviews           0   
3     4.7 out of 5 stars with 6455 ratings   6455 reviews           0   
4    4.6 out of 5 stars with 15030 ratings  15030 reviews           0   
..                                     ...            ...         ...   
153    4.5 out of 5 stars with 443 ratings    443 reviews           0   
154   4.7 out of 5 stars with 7200 ratings   7200 reviews           0   
155    4.5 out of 5 stars with 990 ratings    990 reviews           0   
156    4.2 out of 5 stars with 439 ratings    439 reviews           0   
157   4.6 out of 5 stars with 1890 ratings   1890 reviews           0   

    is_free_shipping is_3day_delivery product_category  \
0                N/A                1        Cleansers   
1                  0                1        Cleansers   
2                N/A                1        Cleansers   
3                N/A                1        Cleansers   
4                N/A                1        Cleansers   
..               ...              ...              ...   
153                0                1        Cleansers   
154                0                1        Cleansers   
155              N/A                1        Cleansers   
156                0                1        Cleansers   
157                0                1        Cleansers   

                                          product_link  
0    https://www.target.com/p/la-roche-posay-toleri...  
1    https://www.target.com/p/geologie-clear-system...  
2    https://www.target.com/p/cerave-foaming-face-w...  
3    https://www.target.com/p/la-roche-posay-toleri...  
4    https://www.target.com/p/cerave-face-wash-hydr...  
..                                                 ...  
153  https://www.target.com/p/nivea-men-maximum-hyd...  
154  https://www.target.com/p/clinique-all-about-cl...  
155  https://www.target.com/p/pixi-skintreats-glow-...  
156  https://www.target.com/p/neutrogena-makeup-rem...  
157  https://www.target.com/p/tula-skincare-so-poli...  

[158 rows x 11 columns]

In [24]:
df1.to_csv('product_data.csv', index=False)

# Page 7-12

In [31]:
import time
import random
import re
import datetime
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

# ========== 1. User-Agent 列表（仅桌面端 Chrome） ==========
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6408.21 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_5_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6408.21 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6408.21 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6367.83 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_5_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6367.83 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; rv:124.0) Gecko/20100101 Chrome/124.0.6367.83 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; rv:123.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.113 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_4_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.113 Safari/537.36",
    "Mozilla/5.0 (X11; openSUSE; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6261.140 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6261.139 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_4_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.170 Safari/537.36",
    "Mozilla/5.0 (X11; Linux Mint x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.172 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6099.229 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_3_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6099.229 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Manjaro) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.6045.193 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.5993.122 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_3_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.5993.122 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Debian) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.5938.151 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.5845.123 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.5845.123 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Kali) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.5790.172 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.5790.172 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_6_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.145 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Solus) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.5672.95 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.5615.138 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_5_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.5615.138 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.5563.148 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.5481.77 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_6_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.5481.77 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Gentoo) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.5414.120 Safari/537.36"
]


random_user_agent = random.choice(USER_AGENTS)

# ========== 2. 初始化 WebDriver ==========
service = Service("/Users/zhuxiaowen/Desktop/chromedriver-mac-arm64/chromedriver")
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument(f"user-agent={random_user_agent}")
options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(service=service, options=options)
print(f"✅ 启动 WebDriver, 使用UA: {random_user_agent}")

# ========== 3. 访问页面 ==========
base_url = "https://www.target.com/c/facial-cleansers-skin-care-beauty/-/N-5xtzi?moveTo=product-list-grid&Nao=144"
driver.get(base_url)
time.sleep(random.uniform(5, 10))

# ========== 4. 修改点①：等待商品卡片加载完成 ==========
def wait_until_products_loaded(min_count=25, timeout=15):
    try:
        WebDriverWait(driver, timeout).until(
            lambda d: len(d.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")) >= min_count
        )
    except TimeoutException:
        print(f"⚠️ 超时未加载出 {min_count} 个商品，当前页可能不完整")

# ========== 5. 修改点②：强化滚动逻辑，直到商品数量不再增加 ==========
def scroll_and_wait():
    last_height = driver.execute_script("return document.body.scrollHeight")
    prev_count = 0

    for i in range(30):
        print(f"🔄 滚动页面: 第 {i+1} 次")
        driver.execute_script("window.scrollBy(0, 1200);")
        time.sleep(random.uniform(2, 4))

        products = driver.find_elements(By.XPATH, "//a[@data-test='product-title']")
        if len(products) > prev_count:
            prev_count = len(products)
        else:
            print("✅ 页面懒加载完成")
            break

        try:
            if products:
                ActionChains(driver).move_to_element(products[-1]).perform()
        except:
            pass

    time.sleep(3)

# ...前面的 import 和设置不变...

# ========== 🔧 修改点①：记录每一页的首个商品链接 ==========
visited_first_links = set()

product_list2 = []

def scrape_products():
    global product_list2, visited_first_links
    scroll_and_wait()
    wait_until_products_loaded()

    products = driver.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")
    print(f"🔍 当前页面商品数: {len(products)}")

    if not products:
        print("⚠️ 本页无商品，跳过")
        return

    # 🔧 修改点②：判断是否回到了已访问页面（用首个商品链接判重）
    try:
        first_link = products[0].find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
    except:
        first_link = None

    if first_link in visited_first_links:
        print("🔁 检测到页面内容重复，疑似回到了第一页，停止抓取")
        raise StopIteration
    visited_first_links.add(first_link)

    for product in products:
        try:
            title_element = product.find_element(By.XPATH, ".//a[@data-test='product-title']//div")
            name = title_element.text.strip()
        except NoSuchElementException:
            continue

        try:
            brand = product.find_element(By.XPATH, ".//a[@data-test='@web/ProductCard/ProductCardBrandAndRibbonMessage/brand']").text.strip()
        except:
            brand = "N/A"

        try:
            price = product.find_element(By.XPATH, ".//span[@data-test='current-price']").text.strip()
        except:
            price = "N/A"

        try:
            unit_price = product.find_element(By.XPATH, ".//span[@data-test='unit-price']").text.strip()
        except:
            unit_price = "N/A"

        try:
            rating = product.find_element(By.XPATH, ".//span[@data-test='ratings']").text.strip()
        except:
            rating = "N/A"

        try:
            reviews = product.find_element(By.XPATH, ".//span[@data-test='rating-count']").text.strip()
        except:
            reviews = "0 reviews"

        try:
            product.find_element(By.XPATH, ".//div[@data-test='comparison-price']")
            is_on_sale = 1
        except:
            is_on_sale = 0

        try:
            if " - " in price:
                is_free_shipping = "N/A"
            else:
                price_value = float(price.replace("$", ""))
                is_free_shipping = 1 if price_value >= 35 else 0
        except:
            is_free_shipping = "N/A"

        try:
            shipping_info = product.find_element(By.XPATH, ".//div[@data-test='LPFulfillmentSectionShippingFA_standardShippingMessage']").text
            match = re.search(r"arrives\s+\w+,\s+([A-Za-z]+ \d+)", shipping_info)
            if match:
                delivery_str = match.group(1)
                delivery_date = datetime.datetime.strptime(f"{delivery_str} {datetime.datetime.now().year}", "%b %d %Y").date()
                days_diff = (delivery_date - datetime.datetime.now().date()).days
                is_3day_delivery = 1 if 0 <= days_diff <= 3 else 0
            else:
                is_3day_delivery = "N/A"
        except:
            is_3day_delivery = "N/A"

        try:
            link = product.find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
        except:
            link = "N/A"

        product_list2.append({
            "product_name": name,
            "brand": brand,
            "current_price": price,
            "unit_price": unit_price,
            "star_rating": rating,
            "review_counts": reviews,
            "is_on_sale": is_on_sale,
            "is_free_shipping": is_free_shipping,
            "is_3day_delivery": is_3day_delivery,
            "product_category": "Cleansers",
            "product_link": link
        })

# 翻页逻辑（原样）
def go_to_next_page():
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@data-test='next']"))
        )
        driver.execute_script("arguments[0].click();", next_button)
        print("✅ 点击下一页")
        time.sleep(random.uniform(8, 12))
        return True
    except:
        print("❌ 没有找到下一页按钮，结束爬取")
        return False

# ========== 主循环（加异常捕捉 StopIteration） ==========
max_pages = 6
try:
    for page_num in range(max_pages):
        print(f"\n📄 正在爬取第 {page_num + 1} 页...")
        scrape_products()
        if not go_to_next_page():
            break
except StopIteration:
    print("⛔ 检测到重复页，主动中止爬虫")

driver.quit()
print("✅ WebDriver 关闭")

print(f"📦 共抓取商品（去重后）：{len(product_list2)} 条")

✅ 启动 WebDriver, 使用UA: Mozilla/5.0 (Macintosh; Intel Mac OS X 12_4_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.113 Safari/537.36

📄 正在爬取第 1 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页

📄 正在爬取第 2 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页

📄 正在爬取第 3 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页

📄 正在爬取第 4 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页

📄 正在爬取第 5 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页

📄 正在爬取第 6 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页
✅ WebDriver 关闭
📦 共抓取商品（去重后）：161 条


In [32]:
df2 = pd.DataFrame(product_list2)
df2

product_name                 brand  \
0    Almay Biodegradable Micellar Makeup Remover Cl...                 Almay   
1    Aveeno Clear Complexion Foaming Oil-Free Facia...                Aveeno   
2               Eminence Stone Crop Cleansing Oil 5 oz              Eminence   
3    All In One Micellar Face Cleansing Water - 13....                 up&up   
4         Naturium Mandelic Topical Acid 12% - 1 fl oz              Naturium   
..                                                 ...                   ...   
156               Epionce Gentle Foaming Cleanser 6 oz               Epionce   
157  Set The Tone Hydrating Rose Mist Toner, Facial...  The Organic Skin Co.   
158  Dear, Klairs K-Beauty Skincare Gentle Black De...                Klairs   
159                    The People's Wash Cleansing Oil             Face Haus   
160  Dear, Klairs K-Beauty Skincare, Supple Prepara...                Klairs   

    current_price           unit_price                           star_rating  \
0           $7.19        ($0.29/count)   4.5 out of 5 stars with 904 ratings   
1           $8.89  ($1.48/fluid ounce)  4.4 out of 5 stars with 2444 ratings   
2          $41.19                  N/A       3 out of 5 stars with 2 ratings   
3           $6.99        ($0.52/ounce)   3.7 out of 5 stars with 324 ratings   
4          $20.00                  N/A    4.5 out of 5 stars with 74 ratings   
..            ...                  ...                                   ...   
156        $32.66                  N/A       1 out of 5 stars with 1 ratings   
157        $24.99                  N/A      5 out of 5 stars with 29 ratings   
158        $23.99                  N/A       4 out of 5 stars with 4 ratings   
159        $25.00                  N/A     3.7 out of 5 stars with 3 ratings   
160        $23.99                  N/A    4.8 out of 5 stars with 31 ratings   

    review_counts  is_on_sale is_free_shipping is_3day_delivery  \
0     904 reviews           0                0                1   
1    2444 reviews           0                0                1   
2       2 reviews           1                1                0   
3     324 reviews           0                0                1   
4      74 reviews           0                0                1   
..            ...         ...              ...              ...   
156      1 review           1                0                0   
157    29 reviews           1                0                0   
158     4 reviews           0                0                1   
159     3 reviews           0                0                0   
160    31 reviews           0                0                1   

    product_category                                       product_link  
0          Cleansers  https://www.target.com/p/almay-biodegradable-m...  
1          Cleansers  https://www.target.com/p/aveeno-clear-complexi...  
2          Cleansers  https://www.target.com/p/eminence-stone-crop-c...  
3          Cleansers  https://www.target.com/p/all-in-one-micellar-f...  
4          Cleansers  https://www.target.com/p/naturium-mandelic-top...  
..               ...                                                ...  
156        Cleansers  https://www.target.com/p/epionce-gentle-foamin...  
157        Cleansers  https://www.target.com/p/set-the-tone-hydratin...  
158        Cleansers  https://www.target.com/p/dear-klairs-k-beauty-...  
159        Cleansers  https://www.target.com/p/the-people-s-wash-cle...  
160        Cleansers  https://www.target.com/p/dear-klairs-k-beauty-...  

[161 rows x 11 columns]

In [33]:
df2.to_csv('product_data2.csv', index=False)

# Page 13-17

In [37]:
import time
import random
import re
import datetime
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

# ========== 1. User-Agent 列表（仅桌面端 Chrome） ==========
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.6423.12 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_5_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.6423.10 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Arch) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6408.45 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6408.38 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_6_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6367.98 Safari/537.36",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6367.92 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.115 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_1_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.111 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Void Linux) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6261.143 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; rv:122.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6261.137 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_2_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.198 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Garuda) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.193 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:120.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6099.243 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_7_10) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6099.238 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Pop!_OS) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.6045.212 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.6045.206 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.5993.130 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; MX Linux) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.5938.157 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.5845.147 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.5790.178 Safari/537.36"
]



random_user_agent = random.choice(USER_AGENTS)

# ========== 2. 初始化 WebDriver ==========
service = Service("/Users/zhuxiaowen/Desktop/chromedriver-mac-arm64/chromedriver")
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument(f"user-agent={random_user_agent}")
options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(service=service, options=options)
print(f"✅ 启动 WebDriver, 使用UA: {random_user_agent}")

# ========== 3. 访问页面 ==========
base_url = "https://www.target.com/c/facial-cleansers-skin-care-beauty/-/N-5xtzi?moveTo=product-list-grid&Nao=288"
driver.get(base_url)
time.sleep(random.uniform(5, 10))

# ========== 4. 修改点①：等待商品卡片加载完成 ==========
def wait_until_products_loaded(min_count=25, timeout=15):
    try:
        WebDriverWait(driver, timeout).until(
            lambda d: len(d.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")) >= min_count
        )
    except TimeoutException:
        print(f"⚠️ 超时未加载出 {min_count} 个商品，当前页可能不完整")

# ========== 5. 修改点②：强化滚动逻辑，直到商品数量不再增加 ==========
def scroll_and_wait():
    last_height = driver.execute_script("return document.body.scrollHeight")
    prev_count = 0

    for i in range(30):
        print(f"🔄 滚动页面: 第 {i+1} 次")
        driver.execute_script("window.scrollBy(0, 1200);")
        time.sleep(random.uniform(2, 4))

        products = driver.find_elements(By.XPATH, "//a[@data-test='product-title']")
        if len(products) > prev_count:
            prev_count = len(products)
        else:
            print("✅ 页面懒加载完成")
            break

        try:
            if products:
                ActionChains(driver).move_to_element(products[-1]).perform()
        except:
            pass

    time.sleep(3)

# ...前面的 import 和设置不变...

# ========== 🔧 修改点①：记录每一页的首个商品链接 ==========
visited_first_links = set()

product_list3 = []

def scrape_products():
    global product_list3, visited_first_links
    scroll_and_wait()
    wait_until_products_loaded()

    products = driver.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")
    print(f"🔍 当前页面商品数: {len(products)}")

    if not products:
        print("⚠️ 本页无商品，跳过")
        return

    # 🔧 修改点②：判断是否回到了已访问页面（用首个商品链接判重）
    try:
        first_link = products[0].find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
    except:
        first_link = None

    if first_link in visited_first_links:
        print("🔁 检测到页面内容重复，疑似回到了第一页，停止抓取")
        raise StopIteration
    visited_first_links.add(first_link)

    for product in products:
        try:
            title_element = product.find_element(By.XPATH, ".//a[@data-test='product-title']//div")
            name = title_element.text.strip()
        except NoSuchElementException:
            continue

        try:
            brand = product.find_element(By.XPATH, ".//a[@data-test='@web/ProductCard/ProductCardBrandAndRibbonMessage/brand']").text.strip()
        except:
            brand = "N/A"

        try:
            price = product.find_element(By.XPATH, ".//span[@data-test='current-price']").text.strip()
        except:
            price = "N/A"

        try:
            unit_price = product.find_element(By.XPATH, ".//span[@data-test='unit-price']").text.strip()
        except:
            unit_price = "N/A"

        try:
            rating = product.find_element(By.XPATH, ".//span[@data-test='ratings']").text.strip()
        except:
            rating = "N/A"

        try:
            reviews = product.find_element(By.XPATH, ".//span[@data-test='rating-count']").text.strip()
        except:
            reviews = "0 reviews"

        try:
            product.find_element(By.XPATH, ".//div[@data-test='comparison-price']")
            is_on_sale = 1
        except:
            is_on_sale = 0

        try:
            if " - " in price:
                is_free_shipping = "N/A"
            else:
                price_value = float(price.replace("$", ""))
                is_free_shipping = 1 if price_value >= 35 else 0
        except:
            is_free_shipping = "N/A"

        try:
            shipping_info = product.find_element(By.XPATH, ".//div[@data-test='LPFulfillmentSectionShippingFA_standardShippingMessage']").text
            match = re.search(r"arrives\s+\w+,\s+([A-Za-z]+ \d+)", shipping_info)
            if match:
                delivery_str = match.group(1)
                delivery_date = datetime.datetime.strptime(f"{delivery_str} {datetime.datetime.now().year}", "%b %d %Y").date()
                days_diff = (delivery_date - datetime.datetime.now().date()).days
                is_3day_delivery = 1 if 0 <= days_diff <= 3 else 0
            else:
                is_3day_delivery = "N/A"
        except:
            is_3day_delivery = "N/A"

        try:
            link = product.find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
        except:
            link = "N/A"

        product_list3.append({
            "product_name": name,
            "brand": brand,
            "current_price": price,
            "unit_price": unit_price,
            "star_rating": rating,
            "review_counts": reviews,
            "is_on_sale": is_on_sale,
            "is_free_shipping": is_free_shipping,
            "is_3day_delivery": is_3day_delivery,
            "product_category": "Cleansers",
            "product_link": link
        })

# 翻页逻辑（原样）
def go_to_next_page():
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@data-test='next']"))
        )
        driver.execute_script("arguments[0].click();", next_button)
        print("✅ 点击下一页")
        time.sleep(random.uniform(8, 12))
        return True
    except:
        print("❌ 没有找到下一页按钮，结束爬取")
        return False

# ========== 主循环（加异常捕捉 StopIteration） ==========
max_pages = 5
try:
    for page_num in range(max_pages):
        print(f"\n📄 正在爬取第 {page_num + 1} 页...")
        scrape_products()
        if not go_to_next_page():
            break
except StopIteration:
    print("⛔ 检测到重复页，主动中止爬虫")

driver.quit()
print("✅ WebDriver 关闭")

print(f"📦 共抓取商品（去重后）：{len(product_list3)} 条")

✅ 启动 WebDriver, 使用UA: Mozilla/5.0 (X11; Linux x86_64; Garuda) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.193 Safari/537.36

📄 正在爬取第 1 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页

📄 正在爬取第 2 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页

📄 正在爬取第 3 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页

📄 正在爬取第 4 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页

📄 正在爬取第 5 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页
✅ WebDriver 关闭
📦 共抓取商品（去重后）：93 条


In [2]:
df3 = pd.DataFrame(product_list3)
df3

NameError: name 'pd' is not defined

In [40]:
df3.to_csv('product_data3.csv', index=False)

# Page 18-23

In [1]:
import time
import random
import re
import datetime
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

# ========== 1. User-Agent 列表（仅桌面端 Chrome） ==========
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6408.21 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_5_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6408.21 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6408.21 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6367.83 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_5_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6367.83 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; rv:124.0) Gecko/20100101 Chrome/124.0.6367.83 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; rv:123.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.113 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_4_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.113 Safari/537.36",
    "Mozilla/5.0 (X11; openSUSE; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6261.140 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6261.139 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_4_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.170 Safari/537.36",
    "Mozilla/5.0 (X11; Linux Mint x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.172 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6099.229 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_3_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6099.229 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Manjaro) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.6045.193 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.5993.122 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_3_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.5993.122 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Debian) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.5938.151 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.5845.123 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.5845.123 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Kali) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.5790.172 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.5790.172 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_6_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.145 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Solus) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.5672.95 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.5615.138 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_5_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.5615.138 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.5563.148 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.5481.77 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_6_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.5481.77 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Gentoo) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.5414.120 Safari/537.36"
]


random_user_agent = random.choice(USER_AGENTS)

# ========== 2. 初始化 WebDriver ==========
service = Service("/Users/zhuxiaowen/Desktop/chromedriver-mac-arm64/chromedriver")
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument(f"user-agent={random_user_agent}")
options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(service=service, options=options)
print(f"✅ 启动 WebDriver, 使用UA: {random_user_agent}")

# ========== 3. 访问页面 ==========
base_url = "https://www.target.com/c/facial-cleansers-skin-care-beauty/-/N-5xtzi?moveTo=product-list-grid&Nao=408"
driver.get(base_url)
time.sleep(random.uniform(5, 10))

# ========== 4. 修改点①：等待商品卡片加载完成 ==========
def wait_until_products_loaded(min_count=25, timeout=15):
    try:
        WebDriverWait(driver, timeout).until(
            lambda d: len(d.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")) >= min_count
        )
    except TimeoutException:
        print(f"⚠️ 超时未加载出 {min_count} 个商品，当前页可能不完整")

# ========== 5. 修改点②：强化滚动逻辑，直到商品数量不再增加 ==========
def scroll_and_wait():
    last_height = driver.execute_script("return document.body.scrollHeight")
    prev_count = 0

    for i in range(30):
        print(f"🔄 滚动页面: 第 {i+1} 次")
        driver.execute_script("window.scrollBy(0, 1200);")
        time.sleep(random.uniform(2, 4))

        products = driver.find_elements(By.XPATH, "//a[@data-test='product-title']")
        if len(products) > prev_count:
            prev_count = len(products)
        else:
            print("✅ 页面懒加载完成")
            break

        try:
            if products:
                ActionChains(driver).move_to_element(products[-1]).perform()
        except:
            pass

    time.sleep(3)

# ...前面的 import 和设置不变...

# ========== 🔧 修改点①：记录每一页的首个商品链接 ==========
visited_first_links = set()

product_list5 = []

def scrape_products():
    global product_list5, visited_first_links
    scroll_and_wait()
    wait_until_products_loaded()

    products = driver.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")
    print(f"🔍 当前页面商品数: {len(products)}")

    if not products:
        print("⚠️ 本页无商品，跳过")
        return

    # 🔧 修改点②：判断是否回到了已访问页面（用首个商品链接判重）
    try:
        first_link = products[0].find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
    except:
        first_link = None

    if first_link in visited_first_links:
        print("🔁 检测到页面内容重复，疑似回到了第一页，停止抓取")
        raise StopIteration
    visited_first_links.add(first_link)

    for product in products:
        try:
            title_element = product.find_element(By.XPATH, ".//a[@data-test='product-title']//div")
            name = title_element.text.strip()
        except NoSuchElementException:
            continue

        try:
            brand = product.find_element(By.XPATH, ".//a[@data-test='@web/ProductCard/ProductCardBrandAndRibbonMessage/brand']").text.strip()
        except:
            brand = "N/A"

        try:
            price = product.find_element(By.XPATH, ".//span[@data-test='current-price']").text.strip()
        except:
            price = "N/A"

        try:
            unit_price = product.find_element(By.XPATH, ".//span[@data-test='unit-price']").text.strip()
        except:
            unit_price = "N/A"

        try:
            rating = product.find_element(By.XPATH, ".//span[@data-test='ratings']").text.strip()
        except:
            rating = "N/A"

        try:
            reviews = product.find_element(By.XPATH, ".//span[@data-test='rating-count']").text.strip()
        except:
            reviews = "0 reviews"

        try:
            product.find_element(By.XPATH, ".//div[@data-test='comparison-price']")
            is_on_sale = 1
        except:
            is_on_sale = 0

        try:
            if " - " in price:
                is_free_shipping = "N/A"
            else:
                price_value = float(price.replace("$", ""))
                is_free_shipping = 1 if price_value >= 35 else 0
        except:
            is_free_shipping = "N/A"

        try:
            shipping_info = product.find_element(By.XPATH, ".//div[@data-test='LPFulfillmentSectionShippingFA_standardShippingMessage']").text
            match = re.search(r"arrives\s+\w+,\s+([A-Za-z]+ \d+)", shipping_info)
            if match:
                delivery_str = match.group(1)
                delivery_date = datetime.datetime.strptime(f"{delivery_str} {datetime.datetime.now().year}", "%b %d %Y").date()
                days_diff = (delivery_date - datetime.datetime.now().date()).days
                is_3day_delivery = 1 if 0 <= days_diff <= 3 else 0
            else:
                is_3day_delivery = "N/A"
        except:
            is_3day_delivery = "N/A"

        try:
            link = product.find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
        except:
            link = "N/A"

        product_list5.append({
            "product_name": name,
            "brand": brand,
            "current_price": price,
            "unit_price": unit_price,
            "star_rating": rating,
            "review_counts": reviews,
            "is_on_sale": is_on_sale,
            "is_free_shipping": is_free_shipping,
            "is_3day_delivery": is_3day_delivery,
            "product_category": "Cleansers",
            "product_link": link
        })

# 翻页逻辑（原样）
def go_to_next_page():
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@data-test='next']"))
        )
        driver.execute_script("arguments[0].click();", next_button)
        print("✅ 点击下一页")
        time.sleep(random.uniform(8, 12))
        return True
    except:
        print("❌ 没有找到下一页按钮，结束爬取")
        return False

# ========== 主循环（加异常捕捉 StopIteration） ==========
max_pages = 6
try:
    for page_num in range(max_pages):
        print(f"\n📄 正在爬取第 {page_num + 1} 页...")
        scrape_products()
        if not go_to_next_page():
            break
except StopIteration:
    print("⛔ 检测到重复页，主动中止爬虫")

driver.quit()
print("✅ WebDriver 关闭")

print(f"📦 共抓取商品（去重后）：{len(product_list5)} 条")

✅ 启动 WebDriver, 使用UA: Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.5845.123 Safari/537.36

📄 正在爬取第 1 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页

📄 正在爬取第 2 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页

📄 正在爬取第 3 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页

📄 正在爬取第 4 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页

📄 正在爬取第 5 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页

📄 正在爬取第 6 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页
✅ WebDriver 关闭
📦 共抓取商品（去重后）：174 条


In [34]:
df5 = pd.DataFrame(product_list5)
df5.loc[df5['is_3day_delivery'] == 'N/A', 'is_3day_delivery'] = 1
df5.sample(5)

product_name           brand  \
107      ELEMIS Dynamic Resurfacing Facial Wash 6.7 oz          ELEMIS   
138  Formula 10.0.6 One Smooth Operator Facial Clea...  Formula 10.0.6   
131  Tree To Tub Gentle Reusable Makeup Remover Tow...     Tree to Tub   
89       Bulldog Men's Original Face Scrub - 4.2 fl oz         Bulldog   
119  Mad Hippie Jelly Cleanser with Rose, Chamomile...      Mad Hippie   

    current_price           unit_price                          star_rating  \
107        $37.49                  N/A  4.8 out of 5 stars with 572 ratings   
138         $7.99  ($2.35/fluid ounce)   4.7 out of 5 stars with 19 ratings   
131        $13.99                  N/A   4.7 out of 5 stars with 56 ratings   
89          $7.29        ($1.74/ounce)  4.6 out of 5 stars with 125 ratings   
119        $17.99                  N/A   4.9 out of 5 stars with 43 ratings   

    review_counts  is_on_sale is_free_shipping is_3day_delivery  \
107   572 reviews           1                1                0   
138    19 reviews           0                0                1   
131    56 reviews           1                0                0   
89    125 reviews           0                0                1   
119    43 reviews           0                0                0   

    product_category                                       product_link  
107        Cleansers  https://www.target.com/p/elemis-dynamic-resurf...  
138        Cleansers  https://www.target.com/p/formula-10-0-6-one-sm...  
131        Cleansers  https://www.target.com/p/tree-to-tub-gentle-re...  
89         Cleansers  https://www.target.com/p/bulldog-men-s-origina...  
119        Cleansers  https://www.target.com/p/mad-hippie-jelly-clea...

In [21]:
df5.to_csv('product_data5.csv', index=False)


# 汇总前23页

In [42]:
df_combined_cleansers_1_23 = pd.concat([df1, df2, df3,df5], ignore_index=True)
df_combined_cleansers_1_23

product_name              brand  \
0    La Roche Posay Toleriane Purifying Facial Clea...     La Roche Posay   
1    Geologie Clear System Acne Control Face Wash -...           Geologie   
2    CeraVe Foaming Face Wash, Facial Cleanser for ...             CeraVe   
3    La Roche Posay Toleriane Purifying Facial Clea...     La Roche Posay   
4    CeraVe Face Wash, Hydrating Facial Cleanser fo...             CeraVe   
..                                                 ...                ...   
581    Lumene Nordic Hydra Lahde Oat Milk Oil Cleanser             Lumene   
582  Blackwood for Men BioNutrient Foaming Face Was...  Blackwood for Men   
583  CeraVe 2 Pack/16 fl oz Each Foaming Face Wash ...             CeraVe   
584  CeraVe 2 Pack/16fl oz Each Hydrating Face Wash...             CeraVe   
585  Vichy LiftActiv H.A. Anti-Wrinkle Firming Day ...              Vichy   

      current_price            unit_price  \
0    $9.99 - $20.49                   N/A   
1            $18.29   ($4.51/fluid ounce)   
2    $6.49 - $19.69                   N/A   
3    $9.99 - $20.49                   N/A   
4    $6.49 - $19.69                   N/A   
..              ...                   ...   
581          $23.49   ($4.61/fluid ounce)   
582          $12.99   ($2.85/fluid ounce)   
583          $37.42   ($1.17/fluid ounce)   
584          $37.42   ($1.17/fluid ounce)   
585          $44.99  ($26.62/fluid ounce)   

                               star_rating  review_counts  is_on_sale  \
0     4.7 out of 5 stars with 6455 ratings   6455 reviews           0   
1      4.8 out of 5 stars with 240 ratings    240 reviews           0   
2    4.7 out of 5 stars with 13334 ratings  13334 reviews           0   
3     4.7 out of 5 stars with 6455 ratings   6455 reviews           0   
4    4.6 out of 5 stars with 15030 ratings  15030 reviews           0   
..                                     ...            ...         ...   
581      4.6 out of 5 stars with 9 ratings      9 reviews           0   
582     4.1 out of 5 stars with 22 ratings     22 reviews           0   
583   4.7 out of 5 stars with 1027 ratings   1027 reviews           0   
584    4.6 out of 5 stars with 865 ratings    865 reviews           0   
585      2.4 out of 5 stars with 8 ratings      8 reviews           0   

    is_free_shipping is_3day_delivery product_category  \
0                N/A                1        Cleansers   
1                  0                1        Cleansers   
2                N/A                1        Cleansers   
3                N/A                1        Cleansers   
4                N/A                1        Cleansers   
..               ...              ...              ...   
581                0                1        Cleansers   
582                0                1        Cleansers   
583                1                1        Cleansers   
584                1                1        Cleansers   
585                1                1        Cleansers   

                                          product_link  
0    https://www.target.com/p/la-roche-posay-toleri...  
1    https://www.target.com/p/geologie-clear-system...  
2    https://www.target.com/p/cerave-foaming-face-w...  
3    https://www.target.com/p/la-roche-posay-toleri...  
4    https://www.target.com/p/cerave-face-wash-hydr...  
..                                                 ...  
581  https://www.target.com/p/lumene-nordic-hydra-l...  
582  https://www.target.com/p/blackwood-for-men-bio...  
583  https://www.target.com/p/cerave-2-pack-16-fl-o...  
584  https://www.target.com/p/cerave-2-pack-16fl-oz...  
585  https://www.target.com/p/vichy-liftactiv-h-a-a...  

[586 rows x 11 columns]

In [43]:
df_combined_cleansers_1_23.to_csv('cleansers_1_23.csv', index=False)

# 汇总前23页（去除13-17）

In [44]:
df_combined_cleansers_1_12_18_23 = pd.concat([df1, df2,df5], ignore_index=True)
df_combined_cleansers_1_12_18_23

product_name              brand  \
0    La Roche Posay Toleriane Purifying Facial Clea...     La Roche Posay   
1    Geologie Clear System Acne Control Face Wash -...           Geologie   
2    CeraVe Foaming Face Wash, Facial Cleanser for ...             CeraVe   
3    La Roche Posay Toleriane Purifying Facial Clea...     La Roche Posay   
4    CeraVe Face Wash, Hydrating Facial Cleanser fo...             CeraVe   
..                                                 ...                ...   
488    Lumene Nordic Hydra Lahde Oat Milk Oil Cleanser             Lumene   
489  Blackwood for Men BioNutrient Foaming Face Was...  Blackwood for Men   
490  CeraVe 2 Pack/16 fl oz Each Foaming Face Wash ...             CeraVe   
491  CeraVe 2 Pack/16fl oz Each Hydrating Face Wash...             CeraVe   
492  Vichy LiftActiv H.A. Anti-Wrinkle Firming Day ...              Vichy   

      current_price            unit_price  \
0    $9.99 - $20.49                   N/A   
1            $18.29   ($4.51/fluid ounce)   
2    $6.49 - $19.69                   N/A   
3    $9.99 - $20.49                   N/A   
4    $6.49 - $19.69                   N/A   
..              ...                   ...   
488          $23.49   ($4.61/fluid ounce)   
489          $12.99   ($2.85/fluid ounce)   
490          $37.42   ($1.17/fluid ounce)   
491          $37.42   ($1.17/fluid ounce)   
492          $44.99  ($26.62/fluid ounce)   

                               star_rating  review_counts  is_on_sale  \
0     4.7 out of 5 stars with 6455 ratings   6455 reviews           0   
1      4.8 out of 5 stars with 240 ratings    240 reviews           0   
2    4.7 out of 5 stars with 13334 ratings  13334 reviews           0   
3     4.7 out of 5 stars with 6455 ratings   6455 reviews           0   
4    4.6 out of 5 stars with 15030 ratings  15030 reviews           0   
..                                     ...            ...         ...   
488      4.6 out of 5 stars with 9 ratings      9 reviews           0   
489     4.1 out of 5 stars with 22 ratings     22 reviews           0   
490   4.7 out of 5 stars with 1027 ratings   1027 reviews           0   
491    4.6 out of 5 stars with 865 ratings    865 reviews           0   
492      2.4 out of 5 stars with 8 ratings      8 reviews           0   

    is_free_shipping is_3day_delivery product_category  \
0                N/A                1        Cleansers   
1                  0                1        Cleansers   
2                N/A                1        Cleansers   
3                N/A                1        Cleansers   
4                N/A                1        Cleansers   
..               ...              ...              ...   
488                0                1        Cleansers   
489                0                1        Cleansers   
490                1                1        Cleansers   
491                1                1        Cleansers   
492                1                1        Cleansers   

                                          product_link  
0    https://www.target.com/p/la-roche-posay-toleri...  
1    https://www.target.com/p/geologie-clear-system...  
2    https://www.target.com/p/cerave-foaming-face-w...  
3    https://www.target.com/p/la-roche-posay-toleri...  
4    https://www.target.com/p/cerave-face-wash-hydr...  
..                                                 ...  
488  https://www.target.com/p/lumene-nordic-hydra-l...  
489  https://www.target.com/p/blackwood-for-men-bio...  
490  https://www.target.com/p/cerave-2-pack-16-fl-o...  
491  https://www.target.com/p/cerave-2-pack-16fl-oz...  
492  https://www.target.com/p/vichy-liftactiv-h-a-a...  

[493 rows x 11 columns]

In [45]:
df_combined_cleansers_1_12_18_23.to_csv("cleansers_1_12_18_23.csv",index=False)